#### Group B: Filip Baran, Jasper Aden, Pietro Riccardi

This notebook generates data for the upwind + Crank-Nicolson scheme.

Imported libraries

In [51]:
import numpy as np

Parameters and grid setup

In [52]:
# ----------------- parameters -----------------

sigma = 0.001
beta = 100
lamda = 0.6
alpha = 0.1
v0 = 0.01
vg = 0.0
psi = np.inf
D0 = 0.01

# ----------------- grid -----------------

nx, nz, nphi = 70, 20, 20
xmin, xmax   = 0.0, 280
zmin, zmax = -20, 0

x   = np.linspace(xmin, xmax, nx)
z   = np.linspace(zmin, zmax, nz)
phi = np.linspace(0.0, 2*np.pi, nphi, endpoint=False)

dx   = x[1]   - x[0]
dz   = z[1]   - z[0]
dphi = phi[1] - phi[0]

X, Z, PHI = np.meshgrid(x, z, phi, indexing="ij")

Vectorized drift and noise

In [53]:
#drifts in x, z, phi
def mu_x_vec(X, Z, PHI, t):
    """Vectorized x-direction drift"""
    return alpha*np.exp(Z)*np.cos(X - t) + v0*np.sin(PHI) + sigma*(beta + Z)

def mu_z_vec(X, Z, PHI, t):
    """Vectorized z-direction drift"""
    return alpha*np.exp(Z)*np.sin(X - t) + v0*np.cos(PHI) - vg

def mu_phi_vec(X, Z, PHI, t):
    """Vectorized phi-direction drift"""
    return (lamda*alpha*np.exp(Z)*np.cos(X - t + 2*PHI)
            - 1/(2*psi)*np.sin(PHI)
            + sigma/2*(1 + lamda*np.cos(2*PHI)))

#extreme drifts for CFL
def mu_x_max(Z):
    return alpha*np.exp(Z) + v0 + sigma*(beta + Z)

def mu_z_max(Z):
    return alpha*np.exp(Z) + v0 - vg

def mu_phi_max(Z):
    return lamda*alpha*np.exp(Z) - 1/(2*psi) + sigma/2*(1 + lamda)

Crank-Nicolson matrix for diffusion

In [54]:
def build_cn_matrix_phi(Nphi, dt, D0, dphi):
    """
    Build Crank–Nicolson matrix in phi for the diffusion term.
    """
    r = D0 * dt / dphi**2

    main = (1.0 + r) * np.ones(Nphi)
    off  = (-0.5 * r) * np.ones(Nphi - 1)

    A = np.diag(main)
    A += np.diag(off,  1)
    A += np.diag(off, -1)

    # periodic coupling
    A[0,  -1] = -0.5 * r
    A[-1,  0] = -0.5 * r

    return A

def laplacian_phi(f, dphi):
    """Central 2nd derivative in φ with periodic BC (axis=2)."""
    return (np.roll(f, -1, axis=2)
            - 2.0 * f
            + np.roll(f,  1, axis=2)) / dphi**2

Upwind scheme for drift

In [55]:
def drift_operator_upwind(f, X, Z, PHI, t, dx, dz, dphi,
                          bc_x, bc_z, bc_phi='periodic'):
    """
    For x, z:
      - 'closed' : no-flux (zero flux at domain boundaries)
      - 'open'   : outflow-only; outflow allowed, inflow suppressed

    For phi:
      - only 'periodic' is implemented.
    """
    if bc_phi != 'periodic':
        raise ValueError("Only periodic bc_phi is implemented in drift_operator_upwind.")

    Nx, Nz, Nphi = f.shape

    MU_X   = mu_x_vec(X, Z, PHI, t)
    MU_Z   = mu_z_vec(X, Z, PHI, t)
    MU_PHI = mu_phi_vec(X, Z, PHI, t)

    MU_X_int  = 0.5 * (MU_X[1:, :, :] + MU_X[:-1, :, :])
    MU_Xp_int = np.maximum(MU_X_int, 0.0)
    MU_Xm_int = np.minimum(MU_X_int, 0.0)

    F_x = np.zeros((Nx + 1, Nz, Nphi))

    F_x[1:Nx, :, :] = (
        MU_Xp_int * f[:-1, :, :] +
        MU_Xm_int * f[1:,  :, :]
    )

    if bc_x == 'closed':
        F_x[0,  :, :] = 0.0
        F_x[-1, :, :] = 0.0

    elif bc_x == 'open':
        MU_left  = MU_X[0,  :, :]
        MU_right = MU_X[-1, :, :]

        F_x[0,  :, :] = np.minimum(0.0, MU_left)  * f[0,  :, :]

        F_x[-1, :, :] = np.maximum(0.0, MU_right) * f[-1, :, :]

    else:
        raise ValueError(f"Unknown bc_x='{bc_x}'")

    Lx = -(F_x[1:, :, :] - F_x[:-1, :, :]) / dx

    MU_Z_int  = 0.5 * (MU_Z[:, 1:, :] + MU_Z[:, :-1, :])
    MU_Zp_int = np.maximum(MU_Z_int, 0.0)
    MU_Zm_int = np.minimum(MU_Z_int, 0.0)

    F_z = np.zeros((Nx, Nz + 1, Nphi))

    F_z[:, 1:Nz, :] = (
        MU_Zp_int * f[:, :-1, :] +
        MU_Zm_int * f[:, 1:,  :]
    )

    if bc_z == 'closed':
        F_z[:, 0,  :] = 0.0
        F_z[:, -1, :] = 0.0

    elif bc_z == 'open':
        MU_bottom = MU_Z[:, 0,  :]
        MU_top    = MU_Z[:, -1, :]

        F_z[:, 0,  :] = np.minimum(0.0, MU_bottom) * f[:, 0,  :]

        F_z[:, -1, :] = np.maximum(0.0, MU_top)    * f[:, -1, :]

    else:
        raise ValueError(f"Unknown bc_z='{bc_z}'")

    Lz = -(F_z[:, 1:, :] - F_z[:, :-1, :]) / dz

    MU_PHI_int = 0.5 * (MU_PHI[:, :, 1:] + MU_PHI[:, :, :-1])
    MU_PHI_p = np.maximum(MU_PHI_int, 0.0)
    MU_PHI_m = np.minimum(MU_PHI_int, 0.0)

    F_phi = np.zeros((Nx, Nz, Nphi + 1))

    F_phi[:, :, 1:Nphi] = (
        MU_PHI_p * f[:, :, :-1] +
        MU_PHI_m * f[:, :, 1: ]
    )

    MU_phi_b   = 0.5 * (MU_PHI[:, :, 0] + MU_PHI[:, :, -1])
    MU_phi_b_p = np.maximum(MU_phi_b, 0.0)
    MU_phi_b_m = np.minimum(MU_phi_b, 0.0)
    F_bphi = MU_phi_b_p * f[:, :, -1] + MU_phi_b_m * f[:, :, 0]
    F_phi[:, :, 0]  = F_bphi
    F_phi[:, :, -1] = F_bphi

    Lphi = -(F_phi[:, :, 1:] - F_phi[:, :, :-1]) / dphi

    return Lx + Lz + Lphi

Solving Fokker-Planck

In [56]:
def fokker_planck_step_cn(
    f, X, Z, PHI,
    t, dt, dx, dz, dphi,
    A_phi_inv, D0,
    *,
    bc_x, bc_z, bc_phi
):
    Nx, Nz, Nphi = f.shape

    # Drift part at time n
    L_drift_f = drift_operator_upwind(
        f, X, Z, PHI, t, dx, dz, dphi,
        bc_x=bc_x, bc_z=bc_z, bc_phi=bc_phi
    )

    # phi-Laplacian at time n
    lap_f = laplacian_phi(f, dphi)

    # Build RHS for CN solve in phi
    RHS = f + dt * L_drift_f + 0.5 * D0 * dt * lap_f

    f_new = np.empty_like(f)
    for i in range(Nx):
        for j in range(Nz):
            f_new[i, j, :] = A_phi_inv @ RHS[i, j, :]

    return f_new


def solve_fokker_planck_cn(
    f0, t_array, x_grid, z_grid, phi_grid, D0,
    *,
    bc_x, bc_z, bc_phi,
    verbose=True
):
    """
    Time integration using Crank–Nicolson in phi + upwind drift.
    bc_x, bc_z ∈ {'open', 'closed'}
    """
    Nt = len(t_array)
    Nx, Nz, Nphi = f0.shape

    dx   = x_grid[1] - x_grid[0]
    dz   = z_grid[1] - z_grid[0]
    dphi = phi_grid[1] - phi_grid[0]
    dt   = t_array[1] - t_array[0]

    # Meshgrids
    X, Z, PHI = np.meshgrid(x_grid, z_grid, phi_grid, indexing='ij')

    # Precompute CN matrix and its inverse in phi
    A_phi     = build_cn_matrix_phi(Nphi, dt, D0, dphi)
    A_phi_inv = np.linalg.inv(A_phi)

    solution = np.zeros((Nt, Nx, Nz, Nphi))
    solution[0] = f0.copy()

    for n in range(Nt - 1):
        t = t_array[n]
        f = solution[n]

        f_new = fokker_planck_step_cn(
            f, X, Z, PHI,
            t, dt, dx, dz, dphi,
            A_phi_inv, D0,
            bc_x=bc_x, bc_z=bc_z, bc_phi=bc_phi
        )
        solution[n+1] = f_new

        if verbose and (n+1) % max(1, Nt//10) == 0:
            total_prob = np.sum(f_new) * dx * dz * dphi
            print(f"[CN] Step {n+1}/{Nt-1}, t={t_array[n+1]:.4f}, "
                  f"∫f={total_prob:.15f}, min={f_new.min():.3e}, max={f_new.max():.3e}")

    return solution

CFL condition for the advective part

In [57]:
def check_cfl_drift(dt, dx, dz, dphi, Z):
    """
    Check CFL condition for the explicit upwind drift part, using
    *extremised* (analytic upper-bound) drift fields.
    """
    mu_x_ext   = mu_x_max(Z)
    mu_z_ext   = mu_z_max(Z)
    mu_phi_ext = mu_phi_max(Z)

    mu_x_max_val   = np.max(np.abs(mu_x_ext))
    mu_z_max_val   = np.max(np.abs(mu_z_ext))
    mu_phi_max_val = np.max(np.abs(mu_phi_ext))

    Cx      = mu_x_max_val   * dt / dx
    Cz      = mu_z_max_val   * dt / dz
    Cphi    = mu_phi_max_val * dt / dphi
    C_total = Cx + Cz + Cphi

    print("CFL check for explicit drift using extremised drifts:")
    print(f"  max|mu_x|   (bound) ≈ {mu_x_max_val:.3e},   Cx   = {Cx:.3g}")
    print(f"  max|mu_z|   (bound) ≈ {mu_z_max_val:.3e},   Cz   = {Cz:.3g}")
    print(f"  max|mu_phi| (bound) ≈ {mu_phi_max_val:.3e}, Cphi = {Cphi:.3g}")
    print(f"  C_total = Cx + Cz + Cphi = {C_total:.3g}")

    if max(Cx, Cz, Cphi) >= 1.0:
        print("WARNING: at least one Courant number ≥ 1 → drift step may be unstable.")
    elif C_total >= 1.0:
        print("WARNING: sum of Courant numbers ≥ 1 → multi-D explicit upwind may be unstable.")
    else:
        print("CFL looks OK: all C < 1 and C_total < 1 for the drift part.")

    return dict(Cx=Cx, Cz=Cz, Cphi=Cphi, C_total=C_total)


Initial condition and simulation

In [58]:
# ----------------- time grid -----------------
t_final = 3000
dt      = 1
Nt      = int(t_final / dt) + 1
t_array = np.linspace(0.0, t_final, Nt)

# ------------- boundary conditions --------------
# x, z can be 'closed' or 'open'; phi is always 'periodic'

bc_x = 'open'
bc_z = 'open'
bc_phi = 'periodic'

# ------------- checking CFL -------------------
cfl = check_cfl_drift(dt, dx, dz, dphi, Z)

# ------------- initial condition -------------
x0   = 0
z0   = -2
phi0 = -0.5*np.pi

sigma_x   = dx
sigma_z   = dz    
sigma_phi = dphi  

# Gaussian in x, z, and phi
f0 = np.exp(
    -((X - x0)**2       / (2 * sigma_x**2)
      + (Z - z0)**2     / (2 * sigma_z**2)
      + (PHI - phi0)**2 / (2 * sigma_phi**2))
)

# normalise to total probability 1
volume_element = (x[1] - x[0]) * (z[1] - z[0]) * (phi[1] - phi[0])
norm = np.sum(f0) * volume_element
f0 /= norm

# ----------------- run CN solver -----------------

solution_cn = solve_fokker_planck_cn(
    f0, t_array, x, z, phi, D0,
    bc_x=bc_x, bc_z=bc_z, bc_phi=bc_phi,
    verbose=True
)

print('I love the Fokker-Planck equation')

def save_solution(filename, solution, t_array, x, z, phi):
    """
    Save Fokker–Planck solution and grids to a compressed .npz file.
    """
    np.savez_compressed(
        filename,
        solution=solution,
        time=t_array,
        x_grid=x,
        z_grid=z,
        phi_grid=phi,
    )

filename = f"D0={D0}_t={t_final}_x={xmax}_z={zmin}_z0={z0}"
print(filename)
save_solution(filename, solution_cn, t_array, x, z, phi)


CFL check for explicit drift using extremised drifts:
  max|mu_x|   (bound) ≈ 2.100e-01,   Cx   = 0.0518
  max|mu_z|   (bound) ≈ 1.100e-01,   Cz   = 0.104
  max|mu_phi| (bound) ≈ 6.080e-02, Cphi = 0.194
  C_total = Cx + Cz + Cphi = 0.35
CFL looks OK: all C < 1 and C_total < 1 for the drift part.
[CN] Step 300/3000, t=300.0000, ∫f=0.370081902961563, min=1.795e-62, max=6.083e-04
[CN] Step 600/3000, t=600.0000, ∫f=0.232866211926442, min=5.096e-38, max=2.051e-04
[CN] Step 900/3000, t=900.0000, ∫f=0.181906836337120, min=3.015e-26, max=1.035e-04
[CN] Step 1200/3000, t=1200.0000, ∫f=0.154229335050149, min=3.220e-19, max=6.512e-05
[CN] Step 1500/3000, t=1500.0000, ∫f=0.136200895897568, min=2.696e-21, max=4.579e-05
[CN] Step 1800/3000, t=1800.0000, ∫f=0.123180984734487, min=2.690e-24, max=3.430e-05
[CN] Step 2100/3000, t=2100.0000, ∫f=0.112752746940875, min=2.569e-27, max=2.698e-05
[CN] Step 2400/3000, t=2400.0000, ∫f=0.100691717177879, min=2.503e-30, max=2.190e-05
[CN] Step 2700/3000, t=2700.0